STEP 0 : Libraries importation

In [1]:
# STEP 0 : Libraries importation

# Required libraries
!pip install torch transformers datasets tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
import torch
import transformers
import tiktoken
import pandas as pd
import json
import random
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_scheduler
import time

STEP 1 : GPT2 "small" implementation

In [3]:
# Choose GPT-2 model size
MODEL_NAME = "gpt2"  # Options: "gpt2", "gpt2-medium", "gpt2-large"

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Test basic text generation
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage
test_prompt = "How do I make a good coffee?"
print("\n GPT-2 Response:")
print(generate_text(test_prompt))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


 GPT-2 Response:
How do I make a good coffee?

You can make a good coffee by using a variety of different methods.

You can make a good coffee by using a variety of different methods. You can make a good coffee by using a variety of different ingredients.

You can make a good coffee by using a variety of different methods. You can make a good coffee by using a variety of different ingredients.

You can make a good coffee by using a variety of different methods.


STEP 2 :

In [4]:
# SET UP of the instruction dataset

# Load instruction-following dataset
dataset_url = "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json"
dataset_path = "instruction_data.json"

# Download dataset if not available
import urllib.request
urllib.request.urlretrieve(dataset_url, dataset_path)

# Load JSON dataset
with open(dataset_path, "r") as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)
df = df[["instruction", "input", "output"]]  # Keep relevant columns
df.fillna("", inplace=True)  # Ensure no NaN values

print("\n Sample Dataset Entry:")
print(df.head(3))


 Sample Dataset Entry:
                            instruction input  \
0  Give three tips for staying healthy.         
1    What are the three primary colors?         
2    Describe the structure of an atom.         

                                              output  
0  1.Eat a balanced diet and make sure to include...  
1  The three primary colors are red, blue, and ye...  
2  An atom is made up of a nucleus, which contain...  


In [16]:
# First Training/Test

# Remove duplicates
df.drop_duplicates(inplace=True)

# REMARQUE IMPORTANTE : Ici on décide de prendre pour le moment et pour des raisons de temps d'éxécution seulement 10 % du dataset total !

# Shuffle dataset
#df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Load only 10% of the dataset to speed up training
df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)  # Use only 10%

# Split dataset
train_size = int(0.85 * len(df))  # 85% training
val_size = int(0.10 * len(df))  # 10% validation
test_size = len(df) - train_size - val_size  # 5% test

train_df = df[:train_size]
val_df = df[train_size:train_size + val_size]
test_df = df[train_size + val_size:]

# Save datasets as CSV
train_df.to_csv("train.csv", index=False)
val_df.to_csv("validation.csv", index=False)
test_df.to_csv("test.csv", index=False)

print("\n Dataset prepared and split:")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")


 Dataset prepared and split:
Training samples: 4420
Validation samples: 520
Test samples: 260


In [17]:
class InstructionDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=128):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instruction = self.data.iloc[idx]["instruction"]
        input_text = self.data.iloc[idx]["input"]
        response = self.data.iloc[idx]["output"]

        # Format the text for instruction fine-tuning
        formatted_text = f"Instruction: {instruction}\nInput: {input_text}\nResponse: {response}"

        # Ensure padding is applied correctly
        tokens = self.tokenizer(
            formatted_text,
            max_length=self.max_length,
            padding="max_length",  # Ensure padding
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": tokens["input_ids"].squeeze(0),
            "attention_mask": tokens["attention_mask"].squeeze(0),
            "labels": tokens["input_ids"].squeeze(0)
        }

STEP 3 : Instruction Fine-Tuning (Supervised Learning on GPT-2)

In [18]:
# Charger le tokenizer et le modèle GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Définir un token de padding

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Adapter les embeddings
model.to(device)  # Déplacer sur GPU si disponible

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [19]:
from torch.utils.data import DataLoader




# Définir une taille de batch optimisée pour GPU
batch_size = 1  # Commence avec 4, ajuste si besoin

# Charger les datasets
train_dataset = InstructionDataset("train.csv", tokenizer)
val_dataset = InstructionDataset("validation.csv", tokenizer)

# Utiliser pin_memory=True pour accélérer le transfert CPU → GPU
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [20]:
from torch.optim import AdamW
from transformers import get_scheduler
import torch
import torch.nn.functional as F

# Fonction d'entraînement sur GPU avec AMP (Mixed Precision)
def train_model(model, train_loader, val_loader, epochs=3, lr=5e-5, gradient_accumulation_steps=2):
    model.to(device)  # Déplacer le modèle sur GPU
    optimizer = AdamW(model.parameters(), lr=lr)

    # Scheduler pour ajuster le learning rate progressivement
    lr_scheduler = get_scheduler(
        name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs
    )

    # Activation de AMP (mixed precision)
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}...")

        # Mode entraînement
        model.train()
        total_loss = 0

        for step, batch in enumerate(train_loader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()

            # Utiliser AMP pour réduire la mémoire GPU
            with torch.cuda.amp.autocast():
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss / gradient_accumulation_steps  # Gradient accumulation

            # Backpropagation avec AMP
            scaler.scale(loss).backward()

            if (step + 1) % gradient_accumulation_steps == 0:  # Mise à jour des poids
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f" Training Loss: {avg_train_loss:.4f}")

        # Mode validation
        model.eval()
        total_val_loss = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                val_loss = outputs.loss
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        print(f" Validation Loss: {avg_val_loss:.4f}")

    print("\nFine-tuning completed!")

In [21]:
# Run training again
train_model(model, train_loader, val_loader, epochs=5, lr=3e-5, gradient_accumulation_steps=4)

<ipython-input-20-bc714fa9f038>:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
<ipython-input-20-bc714fa9f038>:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/5...
 Training Loss: 0.3737
 Validation Loss: 1.2560
Epoch 2/5...
 Training Loss: 0.3307
 Validation Loss: 1.2336
Epoch 3/5...
 Training Loss: 0.3142
 Validation Loss: 1.2212
Epoch 4/5...
 Training Loss: 0.3004
 Validation Loss: 1.2169
Epoch 5/5...
 Training Loss: 0.2891
 Validation Loss: 1.2162

Fine-tuning completed!


STEP 4 : Evaluation & Improvement

In [22]:
import torch
import math
from transformers import GPT2Tokenizer

# Charger le tokenizer et le modèle fine-tuné
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def evaluate_model(model, val_loader, device):
    """Calcule la perte moyenne et la perplexité sur l'ensemble de validation"""
    model.to(device)
    model.eval()
    total_loss = 0
    num_batches = len(val_loader)

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / num_batches
    perplexity = math.exp(avg_loss)  # Perplexité = e^(loss)
    return avg_loss, perplexity

# Exécuter l'évaluation après l'entraînement
val_loss, perplexity = evaluate_model(model, val_loader, device)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Perplexity: {perplexity:.2f}")


Validation Loss: 1.2162
Perplexity: 3.37


In [23]:
# Tester le modèle et l'efficacité de notre chatbot

def generate_response(prompt, model, tokenizer, max_length=100):
    """Génère une réponse en utilisant le modèle fine-tuné"""
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Exemple de test
test_prompt = "What are the benefits of meditation?"
response = generate_response(test_prompt, model, tokenizer)
print("\nGenerated Response:")
print(response)


Generated Response:
What are the benefits of meditation?
Input: nan
Response: Meditation can help to relax and relax the mind, allowing it to relax and relax the body. It also helps to relax the body's natural rhythm and relax the muscles. Additionally, meditation can help to reduce stress and improve circulation, as well as to reduce the risk of heart disease and stroke.


In [4]:
# Pour voir la qualité du modèle
def calculate_token_accuracy(model, val_loader, device):
    """Calcule l'accuracy token-level sur le jeu de validation"""
    model.to(device)
    model.eval()
    total_correct = 0
    total_tokens = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)  # Prendre le token le plus probable

            # Comparer les tokens prédits avec les labels (ignorer les -100 qui correspondent au padding)
            mask = labels != -100  # Ne pas compter le padding
            correct = (predictions == labels) & mask
            total_correct += correct.sum().item()
            total_tokens += mask.sum().item()

    accuracy = total_correct / total_tokens if total_tokens > 0 else 0
    return accuracy

# Calculer l'accuracy token-level
accuracy = calculate_token_accuracy(model, val_loader, device)
print(f"Token-Level Accuracy: {accuracy * 100:.2f}%")


NameError: name 'model' is not defined

In [5]:
from datasets import load_metric

# Charger le BLEU score de Hugging Face
bleu_metric = load_metric("bleu")

def calculate_bleu(model, val_loader, tokenizer, device):
    """Calcule le BLEU score sur l'ensemble de validation"""
    model.to(device)
    model.eval()
    predictions, references = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            reference_texts = [tokenizer.decode(label, skip_special_tokens=True) for label in batch["labels"]]

            # Générer des prédictions
            output_ids = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
            predicted_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in output_ids]

            predictions.extend(predicted_texts)
            references.extend([[ref] for ref in reference_texts])  # BLEU attend une liste de listes

    # Calculer le BLEU score
    bleu_score = bleu_metric.compute(predictions=predictions, references=references)
    return bleu_score["bleu"]

# Calculer le BLEU score
bleu = calculate_bleu(model, val_loader, tokenizer, device)
print(f"BLEU Score: {bleu * 100:.2f}")


ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.11/dist-packages/datasets/__init__.py)

In [ ]:
from datasets import load_metric

# Charger la métrique ROUGE
rouge_metric = load_metric("rouge")

def calculate_rouge(model, val_loader, tokenizer, device):
    """Calcule le ROUGE score sur le jeu de validation"""
    model.to(device)
    model.eval()
    predictions, references = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            reference_texts = [tokenizer.decode(label, skip_special_tokens=True) for label in batch["labels"]]

            # Générer des réponses
            output_ids = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
            predicted_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in output_ids]

            predictions.extend(predicted_texts)
            references.extend(reference_texts)

    # Calculer le score ROUGE
    rouge_score = rouge_metric.compute(predictions=predictions, references=references)
    return rouge_score

# Calculer le ROUGE score
rouge = calculate_rouge(model, val_loader, tokenizer, device)
print(f"ROUGE Score: {rouge}")

STEP 5 :

In [24]:
def chat_with_model(model, tokenizer):
    """Chat interactif dans le terminal"""
    print("\nWelcome to the GPT-2 Chatbot! Type 'exit' to quit.\n")
    while True:
        prompt = input("You: ")
        if prompt.lower() == "exit":
            print("Goodbye!")
            break

        response = generate_response(prompt, model, tokenizer)
        print(f"GPT-2: {response}\n")

# Lancer le chatbot CLI
chat_with_model(model, tokenizer)


Welcome to the GPT-2 Chatbot! Type 'exit' to quit.

You: hello are you okay ?
GPT-2: hello are you okay ?
Response: I am fine.

You: tell me what is a pokemon
GPT-2: tell me what is a pokemon?
Input: nan
Response: A pokemon is a type of animal that can learn and use movesets. It is a type of animal that can learn and use movesets.

You: exit
Goodbye!


In [1]:
!pip install gradio

In [3]:
import gradio as gr

# Interface Gradio
def gradio_chatbot(prompt):
    return generate_response(prompt, model, tokenizer)

# Créer une UI simple
iface = gr.Interface(
    fn=gradio_chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask me something..."),
    outputs="text",
    title="GPT-2 Chatbot",
    description="This is a fine-tuned GPT-2 chatbot. Type something and see how it responds!"
)

# Lancer l'interface
iface.launch(share=True)  # share=True pour un lien public

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


KeyboardInterrupt: 